# 🔹 Анализ изображений(геоснимков)



### Sentinel‑2
Sentinel‑2 — это серия спутников Copernicus (ESA), которые снимают Землю мультиспектральной камерой и дают данные в видимом диапазоне, ближнем ИК (NIR) и коротковолновом ИК (SWIR).

### Анализ NDVI, NDMI, SWIR
Это расчёт показателей по каналам:
- **NDVI** — индекс “насколько зелёная/живая растительность” (использует Red и NIR).  
- **NDMI** — индекс, связанный с влажностью растительности/поверхности (использует NIR и SWIR).  
- **SWIR** — сами коротковолновые инфракрасные каналы, которые часто используют для оценки влажности, горелых участков и различения типов поверхности.

# 🔹 При работе будем использовать следующие файлы: 

## data.zip / папка `data` (файлы `.hdf5`)
- **Назначение:** содержит исходные данные для обучения и оценки модели.
- **Содержимое:** файлы с мультиспектральными изображениями (массивы пикселей с несколькими каналами/полосами).
- **Использование:** загрузка примеров в даталоадер, обучение, валидация, тестирование, визуальный контроль.

## default_partition.json
- **Назначение:** фиксирует разбиение датасета на обучающую, валидационную и тестовую выборки.
- **Содержимое:** списки идентификаторов примеров (id), распределённых по `train`, `valid`, `test`.
- **Использование:** обеспечивает воспроизводимость экспериментов и корректную оценку качества.

## label_map.json
- **Назначение:** задаёт соответствие между числовыми метками классов и их идентификаторами/названиями.
- **Содержимое:** отображение вида “индекс класса → строковый идентификатор”.
- **Использование:** корректное формирование целевых меток при обучении и интерпретация предсказаний модели.

## band_stats.json
- **Назначение:** предоставляет справочную информацию по спектральным каналам и их значениям.
- **Содержимое:** статистики по каналам (и часто — их порядок/обозначения), необходимые для корректной обработки данных.
- **Использование:** нормализация входов, построение корректных псевдо‑RGB изображений и вычисление спектральных индексов (например, NDVI/NDMI).


# 🔹 1. Проведем анализ данных